# Using notes' topic mixtures as features to develop models for predicting length of MV

In [111]:
import pandas as pd
import numpy as np

## Read data

In [112]:
train_mixture_filename = r'/home/mcb/li_lab/zwen8/data/mimic/d2_phy_nurse_48/infer/train_topics.csv'
train_duration_filename = r'/home/mcb/li_lab/zwen8/data/mimic/d2_phy_nurse_48/train_vent.txt'
valid_mixture_filename = r'/home/mcb/li_lab/zwen8/data/mimic/d2_phy_nurse_48/infer/validation_topics.csv'
valid_duration_filename = r'/home/mcb/li_lab/zwen8/data/mimic/d2_phy_nurse_48/validation_vent.txt'
test_mixture_filename = r'/home/mcb/li_lab/zwen8/data/mimic/d2_phy_nurse_48/infer/test_topics.csv'
test_duration_filename = r'/home/mcb/li_lab/zwen8/data/mimic/d2_phy_nurse_48/test_vent.txt'

In [113]:
train_mixtures = pd.read_csv(train_mixture_filename, header=None)
train_duration = pd.read_csv(train_duration_filename, header=None, sep=' ')
train_duration = train_duration[1].astype(np.float64)
valid_mixtures = pd.read_csv(valid_mixture_filename, header=None)
valid_duration = pd.read_csv(valid_duration_filename, header=None, sep=' ')
valid_duration = valid_duration[1].astype(np.float64)
test_mixtures = pd.read_csv(test_mixture_filename, header=None)
test_duration = pd.read_csv(test_duration_filename, header=None, sep=' ')
test_duration = test_duration[1].astype(np.float64)

In [114]:
train_mixtures.shape

(5831, 50)

In [115]:
train_duration.shape

(5831,)

## Convert to binarized duration

In [116]:
bi_train_duration = (train_duration > 7 * 24).astype(int)
bi_valid_duration = (valid_duration > 7 * 24).astype(int)
bi_test_duration = (test_duration > 7 * 24).astype(int)

## Logistic regression

In [117]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

In [118]:
lr = LogisticRegression(solver='liblinear',  max_iter=200)

In [119]:
lr.fit(train_mixtures, bi_train_duration)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [120]:
valid_predictions_lr = lr.predict(valid_mixtures)

In [121]:
print(roc_auc_score(bi_valid_duration, valid_predictions_lr))
print(classification_report(bi_valid_duration, valid_predictions_lr))

0.6956092916984006
              precision    recall  f1-score   support

           0       0.70      0.86      0.77       404
           1       0.75      0.53      0.62       325

    accuracy                           0.71       729
   macro avg       0.72      0.70      0.70       729
weighted avg       0.72      0.71      0.70       729



## Naive Bayes

In [122]:
from sklearn.naive_bayes import GaussianNB

In [123]:
GNB = GaussianNB()

In [124]:
GNB.fit(train_mixtures, bi_train_duration)

GaussianNB(priors=None, var_smoothing=1e-09)

In [125]:
valid_predictions_GNB = GNB.predict(valid_mixtures)

In [126]:
print(roc_auc_score(bi_valid_duration, valid_predictions_GNB))
print(classification_report(bi_valid_duration, valid_predictions_GNB))

0.5877532368621479
              precision    recall  f1-score   support

           0       0.61      0.91      0.73       404
           1       0.70      0.26      0.38       325

    accuracy                           0.62       729
   macro avg       0.66      0.59      0.56       729
weighted avg       0.65      0.62      0.57       729



## SVM

### Linear SVM

In [127]:
from sklearn.svm import LinearSVC

In [128]:
lsvm = LinearSVC()

In [129]:
lsvm.fit(train_mixtures, bi_train_duration)

/home/mcb/li_lab/zwen8/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [130]:
valid_predictions_lsvm = GNB.predict(valid_mixtures)

In [131]:
print(roc_auc_score(bi_valid_duration, valid_predictions_lsvm))
print(classification_report(bi_valid_duration, valid_predictions_lsvm))

0.5877532368621479
              precision    recall  f1-score   support

           0       0.61      0.91      0.73       404
           1       0.70      0.26      0.38       325

    accuracy                           0.62       729
   macro avg       0.66      0.59      0.56       729
weighted avg       0.65      0.62      0.57       729



### Non-linear SVM

In [132]:
from sklearn.svm import SVC

In [133]:
svm = SVC()

In [134]:
svm.fit(train_mixtures, bi_train_duration)

/home/mcb/li_lab/zwen8/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [135]:
valid_predictions_svm = GNB.predict(valid_mixtures)

In [136]:
print(roc_auc_score(bi_valid_duration, valid_predictions_lsvm))
print(classification_report(bi_valid_duration, valid_predictions_lsvm))

0.5877532368621479
              precision    recall  f1-score   support

           0       0.61      0.91      0.73       404
           1       0.70      0.26      0.38       325

    accuracy                           0.62       729
   macro avg       0.66      0.59      0.56       729
weighted avg       0.65      0.62      0.57       729



## Do Lasso regression

In [137]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [138]:
lasso = Lasso()

In [139]:
lasso.fit(train_mixtures, train_duration)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [140]:
valid_predictions_lasso = lasso.predict(valid_mixtures)

In [141]:
print(mean_squared_error(valid_duration, valid_predictions_lasso))

41810.96397345644


In [142]:
valid_predictions_lasso[0:5]

array([191.45907561, 198.95308671, 141.98132046, 154.46365004,
       222.04591674])

In [143]:
valid_duration.head()

0     68.016667
1    113.550000
2     80.450000
3    281.000000
4    161.000000
Name: 1, dtype: float64

## Grid Search LR
Logistic regression seems to be the best classifier. See how good it can be.

In [199]:
from sklearn.model_selection import GridSearchCV

In [200]:
estimator = LogisticRegression(solver='liblinear')

In [201]:
hyperparams = {
    'C': np.arange(1.0, 1.6, step=0.01),
    'max_iter': [100, 200, 300]
              }

In [202]:
searcher = GridSearchCV(estimator=estimator, n_jobs=12, param_grid=hyperparams, scoring='roc_auc', cv=5)

In [203]:
searcher.fit(train_mixtures, bi_train_duration)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=12,
             param_grid={'C': array([1.  , 1.0...
       1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2 , 1.21,
       1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3 , 1.31, 1.32,
       1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4 , 1.41, 1.42, 1.43,
       1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 1.5 , 1.51, 1.52, 1.53, 1.54

In [204]:
print(searcher.best_params_)

{'C': 1.08, 'max_iter': 100}


In [205]:
print(searcher.best_score_)

0.7509739988131034


In [206]:
valid_predictions_lrbest = searcher.best_estimator_.predict(valid_mixtures)

In [207]:
print(roc_auc_score(bi_valid_duration, valid_predictions_lrbest))
print(classification_report(bi_valid_duration, valid_predictions_lrbest))

0.6956092916984006
              precision    recall  f1-score   support

           0       0.70      0.86      0.77       404
           1       0.75      0.53      0.62       325

    accuracy                           0.71       729
   macro avg       0.72      0.70      0.70       729
weighted avg       0.72      0.71      0.70       729



## Try AdaBoost

In [210]:
from sklearn.ensemble import AdaBoostClassifier

In [274]:
ada = AdaBoostClassifier(base_estimator=LogisticRegression(solver='liblinear', C=1.08, max_iter=100), random_state=1)

In [275]:
ada.fit(train_mixtures, bi_train_duration)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=LogisticRegression(C=1.08, class_weight=None,
                                                     dual=False,
                                                     fit_intercept=True,
                                                     intercept_scaling=1,
                                                     l1_ratio=None,
                                                     max_iter=100,
                                                     multi_class='warn',
                                                     n_jobs=None, penalty='l2',
                                                     random_state=None,
                                                     solver='liblinear',
                                                     tol=0.0001, verbose=0,
                                                     warm_start=False),
                   learning_rate=1.0, n_estimators=50, random_state=1)

In [276]:
valid_predictions_ada = ada.predict(valid_mixtures)

In [277]:
print(roc_auc_score(bi_valid_duration, valid_predictions_ada))
print(classification_report(bi_valid_duration, valid_predictions_ada))

0.6962795125666412
              precision    recall  f1-score   support

           0       0.69      0.88      0.77       404
           1       0.77      0.51      0.62       325

    accuracy                           0.72       729
   macro avg       0.73      0.70      0.70       729
weighted avg       0.73      0.72      0.70       729



## Try MLP (non-linear classifier)

In [208]:
from sklearn.neural_network import MLPClassifier

In [265]:
mlp = MLPClassifier(hidden_layer_sizes=(40, 30),
                   activation='logistic',
                   solver='lbfgs',
                   early_stopping=True,
                   max_iter=300,
                   random_state=1,
                   n_iter_no_change=20)

In [266]:
mlp.fit(train_mixtures, bi_train_duration)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(40, 30), learning_rate='constant',
              learning_rate_init=0.001, max_iter=300, momentum=0.9,
              n_iter_no_change=20, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [267]:
mlp.n_iter_

302

In [268]:
valid_predictions_mlp = mlp.predict(valid_mixtures)

In [269]:
print(roc_auc_score(bi_valid_duration, valid_predictions_mlp))
print(classification_report(bi_valid_duration, valid_predictions_mlp))

0.5814813404417365
              precision    recall  f1-score   support

           0       0.62      0.65      0.64       404
           1       0.54      0.52      0.53       325

    accuracy                           0.59       729
   macro avg       0.58      0.58      0.58       729
weighted avg       0.59      0.59      0.59       729

